In [1]:
import pyconll, keras, pickle, os, random, nltk, datetime, warnings, gc, urllib.request, zipfile
import numpy as np
import math
from scipy.sparse import hstack, vstack
import matplotlib.pyplot as plt
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import FastText
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV, learning_curve, cross_val_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, BatchNormalization, Dropout, Input, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
from numpy.random import seed
from sklearn.model_selection import train_test_split

2023-03-12 22:18:22.833129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 22:18:23.069469: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 22:18:23.069679: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-12 22:18:24.662523: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# Load the directories and files - this is local
# ROOT_DIR = os.path.dirname("/home/chitrang/Documents/CSE-582/")
# POS_DIR = os.path.join(ROOT_DIR, 'dataset')
# pos_train = os.path.join(POS_DIR, "train.txt")

# Load the directories and files - this is relative
pos_train = './../dataset/train.txt'
pos_test = './../dataset/test.txt'

## Data Preparation

In [3]:
def format_data(fname, include_y=True):
    sentences = [] # master list
    with open(fname) as f:
        content = f.readlines()
    
    sentence = [] # local list
    for line in content:
        if line !='\n':
            line = line.strip() # remove leading/trailing spaces
            word = line.split()[0].lower() # get the word
            if include_y:
                pos = ""
                pos = line.split()[1] # get the pos tag
                sentence.append((word, pos)) # create a pair and save to local list
            else:
                sentence.append(word)
        else:
            sentences.append(sentence) # once a \n is detected, append the local sentence to master sentence
            sentence = []
    return sentences

In [4]:
train_sentences = format_data(pos_train)
test_sentences = format_data(pos_test, False)

In [5]:
print("Tagged sentences in train set: ", len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))

Tagged sentences in train set:  8936
Tagged words in train set: 211727


In [6]:
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

def text_sequence(sentences):
    return [[w for w, t in sentence] for sentence in sentences]

def id2word(sentences):
    wordlist = [item for sublist in text_sequence(sentences) for item in sublist]
    id2word = {k:v for k,v in enumerate(wordlist)}
    return id2word

def untag(tagged_sentence):
    return [w for w, _ in tagged_sentence]

def untag_pos(tagged_sentence):
    return [t for _, t in tagged_sentence]

def build_vocab(sentences):
    vocab =set()
    for sentence in sentences:
        for word in untag(sentence):
            vocab.add(word)
    return sorted(list(vocab))

### Load embeddings

In [7]:
embs_path = '/home/chitrang/Downloads/wiki-news-300d-1M.vec'
embeddings = KeyedVectors.load_word2vec_format(embs_path, binary=False)

In [8]:
# Create a dictionary - just for fun!
w2c = dict()
for item in embeddings.key_to_index:
    w2c[item] = embeddings.key_to_index[item]

In [9]:
dim = embeddings.vectors.shape[1]
pad = np.zeros(dim)
np.random.seed(3)
oov = np.random.uniform(-0.25, 0.25, dim)

## Feature Engineering

In [10]:
def features_embs(sentence, index, window=2):
    unknown=0
    vec = np.array([])
    for i in range(index-window, index+window+1):
#         if i < 0:
#             vec = np.append(vec, pad)
#         if i > len(sentence)-1:
#             vec = np.append(vec, pad)
        try:
            vec = np.append(vec, embeddings[sentence[i]])
        except:
            vec = np.append(vec, oov)
            unknown += 1
    return vec, unknown

In [11]:
def features_basic(sentence, index):
    return {
        'nb_terms': len(sentence),        
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'i-1_prefix-3': '' if index == 0 else sentence[index-1][:3],        
        'i-1_suffix-3': '' if index == 0 else sentence[index-1][-3:],        
        'i+1_prefix-3': '' if index == len(sentence) - 1 else sentence[index+1][:3],        
        'i+1_suffix-3': '' if index == len(sentence) - 1 else sentence[index+1][-3:],        
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:],
    }

## Data Transformation

In [12]:
def transform_to_dataset(tagged_sentences, window):
    i=0
    X, y = [], []
    for doc_index, tagged in enumerate(tagged_sentences):
        for index in range(len(tagged)):
            X.append([features_basic(untag(tagged), index),\
                      features_embs(untag(tagged), index, window)[0],\
                     ])
            y.append(tagged[index][1])
            k = features_embs(untag(tagged), index, window)[1]
            i += k
    return X, y, i

In [13]:
def transform_test_sentence(sentence, window):
    X = []
    for index in range(len(sentence)):
            X.append([
                      features_basic(sentence, index),\
                      features_embs(sentence, index, window),\
                     ])
    return X

In [14]:
def vectorize(train, window=2):
    X_train, y_train, unk_tr = transform_to_dataset(train, window=window)
    X_train = [x[1] for x in X_train]
    X_train = np.asarray(X_train)
    return X_train, y_train

In [15]:
X_train, y_train = vectorize(train_sentences)

In [18]:
X_train.shape

(211727, 1500)

In [16]:
classes = sorted(list(set(y_train)))
print(classes)

['#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [17]:
# Dataset split: Train - Validation
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### One hot encoding the labels

In [18]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
# y_val = le.transform(y_val)
y_train = keras.utils.to_categorical(y_train)
# y_val = keras.utils.to_categorical(y_val)

print(y_train.shape)

(211727, 44)


### Train + Val model

In [19]:
# model = Sequential()
# # In the first layer, we specify the input data shape

# model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(y_train.shape[1], activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [20]:
# nb_epoch = 10
# batch_size = 128
# early_stopping = EarlyStopping(monitor = 'val_acc', patience = 5)
# history = model.fit(X_train, y_train,
#                     epochs=nb_epoch,
#                     batch_size=batch_size,
#                     shuffle=True,
#                     validation_data=(X_val, y_val),
#                     verbose=1,
#                     callbacks=[early_stopping])

In [21]:
# X_full_train = vstack((X_train, X_val)).tocsr()
# y_full_train = np.append(y_train, y_val, axis=0)

In [22]:
# X_train.shape

### Full Training Set

In [23]:
model = Sequential()

model.add(Dense(512, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2023-03-12 18:37:28.652704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 18:37:28.652909: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 18:37:28.656450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-12 18:37:28.656703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-12 18:37:28.656750: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               768512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                32832     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 44)                2860      
                                                                 
Total params: 804,204
Trainable params: 804,204
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    verbose=1,
                   ) 

2023-03-12 18:37:33.298584: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1270362000 exceeds 10% of free system memory.
2023-03-12 18:37:35.607576: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 37263952 exceeds 10% of free system memory.


Epoch 1/20
6617/6617 [==============================] - 44s 7ms/step - loss: 0.4496 - accuracy: 0.8721
Epoch 2/20
6617/6617 [==============================] - 35s 5ms/step - loss: 0.2327 - accuracy: 0.9296
Epoch 3/20
6617/6617 [==============================] - 52s 8ms/step - loss: 0.1982 - accuracy: 0.9392
Epoch 4/20
6617/6617 [==============================] - 42s 6ms/step - loss: 0.1774 - accuracy: 0.9453
Epoch 5/20
6617/6617 [==============================] - 39s 6ms/step - loss: 0.1605 - accuracy: 0.9501
Epoch 6/20
6617/6617 [==============================] - 45s 7ms/step - loss: 0.1499 - accuracy: 0.9538
Epoch 7/20
6617/6617 [==============================] - 44s 7ms/step - loss: 0.1432 - accuracy: 0.9557
Epoch 8/20
6617/6617 [==============================] - 49s 7ms/step - loss: 0.1347 - accuracy: 0.9585
Epoch 9/20
6617/6617 [==============================] - 51s 8ms/step - loss: 0.1264 - accuracy: 0.9608
Epoch 10/20
6617/6617 [==============================] - 51s 8ms/step - l

### Save and Load Models

In [25]:
# model.save("mlp_model.h5", save_format='h5')

In [15]:
model = keras.models.load_model("mlp_model.h5")

2023-03-12 22:21:45.840530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 22:21:45.840723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 22:21:45.840777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-12 22:21:45.840814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-12 22:21:45.840847: W tensorflow/c

In [27]:
# dim = embeddings.vectors.shape[1]
# pad = np.zeros(dim)

In [18]:
preprocessed_test_data = []
# Embed test data
def embed_test_sentences(sentence):
    X_embs = [x[1][0] for x in sentence]
    X_embs = np.asarray(X_embs)
    return X_embs

# Preprocess Unlabelled Data
def preprocess_unlabelled_test_data(test_sentences):
    for sentence in test_sentences:
        sentence = transform_test_sentence(sentence, 2)
        embedded = embed_test_sentences(sentence)
        preprocessed_test_data.append(embedded)
        
def generate_labelled_data(file_name):
    f = open(file_name, "w")
    for sentence in predicted_data:
        for word, pos in sentence:
            f.write(f"{word} {pos}\n")
        f.write(f"\n")
    f.close()

In [19]:
preprocess_unlabelled_test_data(test_sentences)

In [20]:
predicted_data = []
arg_max_dict = []
def test_set_predictions(preprocessed_test_data, test_sentences):
    for sentence in preprocessed_test_data:
        predict_x=model.predict(sentence, batch_size=1, verbose=0) 
        predict_x = np.argmax(predict_x, axis=1)
        arg_max_dict.append(predict_x)
        
    for index in range(len(test_sentences)):
        predicted_sen = list(zip(test_sentences[index], le.inverse_transform(arg_max_dict[index])))
        predicted_data.append(predicted_sen)


In [ ]:
test_set_predictions(preprocessed_test_data, test_sentences)

In [37]:
generate_labelled_data('./../Labelled_outputs/mlp_labelled_best.txt')